<a href="https://colab.research.google.com/github/TNH510/Artificial-Intelligence/blob/main/Con_lac_nguoc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#fuzzy logic
#Tip controller

In [3]:
pip install scikit-fuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 KB 32.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894088 sha256=8d93ecfdaf4fa2b6122eb2310cb85d5dc1fef9de6a67a0d53c18cd9cccacb00d
  Stored in directory: /root/.cache/pip/wheels/2c/04/80/7eefb1a2de7d36aefd06432fab2a1486caf0a0596a7067391a
Successfully built scikit-fuzzy


In [4]:
import numpy as np
import skfuzzy as fuzzy
from skfuzzy import control as ctrl

In [5]:
universe = np.linspace(-90,90,61) #0 --> 10, 61 points
universe1 = np.linspace(-45,45,61)
universe2 = np.linspace(-100,100,61)
alpha = ctrl.Antecedent(universe, 'alpha')
d_alpha = ctrl.Antecedent(universe1, 'd_alpha')
f = ctrl.Consequent(universe2, 'f')

In [6]:
alpha['-2'] = fuzzy.gaussmf(alpha.universe,-90,20)
alpha['-1'] = fuzzy.gaussmf(alpha.universe,-45,20)
alpha['0'] = fuzzy.gaussmf(alpha.universe,0,20)
alpha['1'] = fuzzy.gaussmf(alpha.universe,45,20)
alpha['2'] = fuzzy.gaussmf(alpha.universe,90,20)
alpha.view()

In [7]:
d_alpha['-2'] = fuzzy.gaussmf(d_alpha.universe,-45,20)
d_alpha['-1'] = fuzzy.gaussmf(d_alpha.universe,-14,20)
d_alpha['0'] = fuzzy.gaussmf(d_alpha.universe,0,20)
d_alpha['1'] = fuzzy.gaussmf(d_alpha.universe,14,20)
d_alpha['2'] = fuzzy.gaussmf(d_alpha.universe,45,20)
d_alpha.view()

In [8]:
f['-2'] = fuzzy.gaussmf(f.universe,-100,30)
f['-1'] = fuzzy.gaussmf(f.universe,-50,30)
f['0'] = fuzzy.gaussmf(f.universe,0,30)
f['1'] = fuzzy.gaussmf(f.universe,50,30)
f['2'] = fuzzy.gaussmf(f.universe,100,30)
f.view()

In [9]:
#Rule
rule0 = ctrl.Rule(antecedent=((alpha['-2']&d_alpha['-2'])|(alpha['-1']&d_alpha['-2'])|(alpha['-2']&d_alpha['-1'])), consequent= f['-2'])
rule1 = ctrl.Rule(antecedent=((alpha['-1']&d_alpha['-1'])|(alpha['0']&d_alpha['-2'])|(alpha['-1']&d_alpha['0'])|
                              (alpha['0']&d_alpha['-1'])|(alpha['-2']&d_alpha['0'])|(alpha['-2']&d_alpha['1'])|
                              (alpha['1']&d_alpha['-2'])), consequent= f['-1'])
rule2 = ctrl.Rule(antecedent=((alpha['-2']&d_alpha['2'])|(alpha['-1']&d_alpha['1'])|(alpha['0']&d_alpha['0'])|
                              (alpha['1']&d_alpha['-1'])|(alpha['2']&d_alpha['-2'])), consequent= f['0'])
rule3 = ctrl.Rule(antecedent=((alpha['1']&d_alpha['1'])|(alpha['0']&d_alpha['2'])|(alpha['2']&d_alpha['0'])|
                              (alpha['2']&d_alpha['-1'])|(alpha['-1']&d_alpha['2'])|(alpha['1']&d_alpha['0'])|
                              (alpha['0']&d_alpha['1'])), consequent= f['1'])
rule4 = ctrl.Rule(antecedent=((alpha['2']&d_alpha['2'])|(alpha['1']&d_alpha['2'])|(alpha['2']&d_alpha['1'])), consequent= f['2'])

In [14]:
from matplotlib.legend import time
import matplotlib
from skfuzzy.control.controlsystem import ControlSystemSimulation
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

system = ctrl.ControlSystem([rule0, rule1, rule2,rule3,rule4])
sim = ctrl.ControlSystemSimulation(system,flush_after_run= 61*61+1)
upsampled = np.linspace(0,50,61)
x,y = np.meshgrid(upsampled,upsampled)
z = np.zeros_like(x)


In [15]:
for i in range(61):
  for j in range(61):
    sim.input['alpha']= x[i,j]
    sim.input['d_alpha']=y[i,j]
    sim.compute()
    z[i,j]=sim.output['f']
    

<ipython-input-18-afb601a25f01>:4: UserWarning: The following kwargs were not used by contour: 'anpha'
  cset = ax.contourf(x,y,z,zdir='z',offset=9,cmap='viridis',anpha=0.4)
<ipython-input-18-afb601a25f01>:5: UserWarning: The following kwargs were not used by contour: 'anpha'
  cset = ax.contourf(x,y,z,zdir='x',offset=11,cmap='viridis',anpha=0.4)
<ipython-input-18-afb601a25f01>:6: UserWarning: The following kwargs were not used by contour: 'anpha'
  cset = ax.contourf(x,y,z,zdir='y',offset=11,cmap='viridis',anpha=0.4)


In [18]:
fig = plt.figure(figsize =(15,15))
ax = fig.add_subplot(111,projection = '3d')
suf = ax.plot_surface(x,y,z,rstride=1, cstride=1, cmap='viridis',linewidth = 0.4)
cset = ax.contourf(x,y,z,zdir='z',offset=9,cmap='viridis',anpha=0.4)
cset = ax.contourf(x,y,z,zdir='x',offset=11,cmap='viridis',anpha=0.4)
cset = ax.contourf(x,y,z,zdir='y',offset=11,cmap='viridis',anpha=0.4)
ax.view_init(40,200)